# Scikit-learn: dane tekstowe

Aby przeprowadzić machine learning na dokumentach tekstowych, najpierw musimy zmienić treść tekstu na numeryczne wektory cech.

Reprezentacja bag-of-words
Przypisać stałą liczbę całkowitą id do każdego słowa występującego w dowolnym dokumencie zestawu treningowego 
(na przykład przez utworzenie słownika ze słów na indeksy). 
Dla każdego dokumentu j, zliczyć w nim liczbę wystąpień każdego słowa k, zapisując ją w X[k,j]

### Tokenizacja i zamiana tekstów na wektory

Pierwszy krok w analizie kolekcji dokumentów to tokenizacja i filtrowanie stop-wordów (ocjonalnie). Zawarte są w komponencie typu Vectorizer (np CountVectorizer), który buduje słownik mapowania słów na indeksy i przekształca dokumenty w wektory frekwencyjne bag-of-words.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(['dokument pierwszy.', 'To jest tekst drugiego dokumentu'])

Zapoznaj się z innymi sposobami wektoryzacji dostępnymi w scikit-learn:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

### Trenowanie modelu klasyfikatora i predykcja

Dysponując cechami, funkcjami, możemy zacząć trenować klasyfikator, aby spróbować przewidzieć etykietę każdego dokumentu. Przykład dla klasyfikatora naiwnego Bayesa. Scikit-learn zawiera kilka wariantów tego klasyfikatora; Najbardziej odpowiedni do liczenia słów jest wariant MultinomialNB:

In [3]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, Y_train)

NameError: name 'Y_train' is not defined

Wywołanie metody fit() powoduje dopasowanie się modelu do danych treningowych oraz etykiet tych danych.
Obiekt clf jest wytrenowanym modelem klasyfikatora który możemy używać do etykietowania nowych, niewidzianych obserwacji. Robimy to wywołując na nim funkcję predict:

In [4]:
docs_new = ['No hipsters write tweets', 'Machine learning is fun?']
X_new_counts = count_vect.transform(docs_new)
predicted = clf.predict(X_new_tfidf)

NameError: name 'clf' is not defined

### Tworzenie pipelines (potoków?)

Czasem warto dołożyć jeszcze kolejny krok przetwarzania, pomiędzy wektoryzacją a klasyfikatorem. Na przykład transformację TF-IDF, selekcję cech lub redukcję wymiarowości.

Aby uczynić sekwencję transformacji: wektorizer => transformator => klasyfikator łatwiejszą do pracy, scikit-learn udostępnia klasę Pipeline, która zachowuje się jak klasyfikator, jest jednak złożona z większej liczby elementów:

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

Na obiekcie tego typu działają metody fit i predict, typowe dla prostych klasyfikatorów.

In [6]:
_ = text_clf.fit(X_train, Y_train)
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)

NameError: name 'X_train' is not defined

### Ewaluacja modeli

Scikit-learn zapewnia narzędzia do analizy wyników:

In [1]:
from sklearn import metrics
print(metrics.classification_report(Y_true, Y_pred))  # szczegolowa analiza, wszystkie miary
print(metrics.confusion_matrix(Y_true, Y_pred))   # macierz konfuzji / confusion matrix

NameError: name 'Y_true' is not defined

In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('Tweets.csv')
df = df[df['airline_sentiment'] != 'neutral']

len(df)
X = df['text']
y = df['airline_sentiment']

X_train = X[:7000]
X_validate = X[7000:9000]
X_test = X[9000:]


y_train = y[:7000]
y_validate = y[7000:9000]
y_test = y[9000:]


#podmiana na 0/1:
y = y.map({'negative': 0, 'positive': 1}).values

def print_significant_features(pipeline=None, n=20):
    feature_names = pipeline.get_params()['vect'].get_feature_names()
    coefs = []
    try:
        coefs = pipeline.get_params()['clf'].coef_
    except:
        coefs.append(pipeline.get_params()['clf'].feature_importances_)
    print("Total features: {}".format(len(coefs[0])))
    coefs_with_fns = sorted(zip(coefs[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_2, fn_2) in top:
        print( coef_2, str(fn_2) )


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([('vect', TfidfVectorizer(stop_words='english')),
                      ('clf', RandomForestClassifier()),
 ])

text_clf.fit(X_train, y_train)

print_significant_features(text_clf)

Total features: 9389
(0.0, '000ft') (0.070304980091557126, 'thank')
(0.0, '0016') (0.06285133437458762, 'thanks')
(0.0, '01') (0.022707797299446532, 'united')
(0.0, '0162389030167') (0.019523991519922391, 'jetblue')
(0.0, '0162424965446') (0.01941732949794809, 'great')
(0.0, '0162431184663') (0.017811943534343344, 'southwestair')
(0.0, '01pm') (0.014255135738272345, 'love')
(0.0, '04') (0.01018515432025759, 'virginamerica')
(0.0, '05') (0.0098561532038626502, 'awesome')
(0.0, '0510') (0.0089719485287093007, 'amazing')
(0.0, '05am') (0.0089191979065771886, 'best')
(0.0, '05pm') (0.0087705285743912009, 'flight')
(0.0, '0600') (0.0079705583492003344, 'http')
(0.0, '0638') (0.0069644358432084134, 'cancelled')
(0.0, '0985') (0.0064113261615704551, 'good')
(0.0, '0bjnz4eix5') (0.0052116696346120603, 'appreciate')
(0.0, '0ewj7oklji') (0.0048050008753320345, 'thx')
(0.0, '0liwecasoe') (0.0043741624092405841, 'hours')
(0.0, '0prgysvurm') (0.0042515291569005546, 'thnx')
(0.0, '0wbjawx7xd') (0.00

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('Tweets.csv')

In [9]:
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
6,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,570300248553349120,neutral,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,@VirginAmerica Really missed a prime opportuni...,NaN,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,570299953286942721,positive,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",NaN,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,570295459631263746,positive,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


In [10]:
len(df)

14640

In [11]:
df = df[df['airline_sentiment'] != 'neutral']

In [12]:
len(df)

11541

In [13]:
X = df['text']
y = df['airline_sentiment']

X_train = X[:7000]
X_validate = X[7000:9000]
X_test = X[9000:]


y_train = y[:7000]
y_validate = y[7000:9000]
y_test = y[9000:]

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([('vect', TfidfVectorizer(stop_words='english')),
                      ('clf', RandomForestClassifier()),
 ])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [36]:
Y_pred = text_clf.predict(X_test)

In [37]:
from sklearn import metrics
print(metrics.classification_report(y_test, Y_pred))  # szczegolowa analiza, wszystkie miary
print(metrics.confusion_matrix(y_test, Y_pred))

             precision    recall  f1-score   support

   negative       0.93      0.98      0.95      2163
   positive       0.81      0.58      0.67       378

avg / total       0.91      0.92      0.91      2541

[[2111   52]
 [ 159  219]]


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
tfidf = TfidfVectorizer()
art = tfidf.fit_transform(articles)
art

NameError: name 'articles' is not defined